In [ ]:
import numpy as np
import tensorflow as tf
import random, math
import matlab
import matlab.engine as me
import itertools
from myTensorUtil import *
from powMeth import *
from recmagsign import *

In [ ]:
## Hyper-Parameters
n = 4000 # number of samples
d = 20  # input dimension
k = 5   # hidden layer size
kappa = 2 
lr = 0.02
batch_size = 10

In [ ]:
## Activation Function
def phi_s(h):
    return h**2 if h > 0 else 0
phi = np.vectorize(phi_s)

In [ ]:
## Data and Params Generation

gauss_mat_u = np.random.normal(0.0, 1.0 , (d,k))
gauss_mat_v = np.random.normal(0.0, 1.0 , (k,k))

U, temp = np.linalg.qr(gauss_mat_u)
V, temp = np.linalg.qr(gauss_mat_v)

# U, V = np.linalg.qr(gauss_mat_u)
diag = []
v_gt = []
v_choice = [1,-1]
for iter in range(k):
    diag.append(1+1.*iter*(kappa-1)/(k-1))
    v_gt.append(random.choice(v_choice))
    
Sigma = np.diag(diag)
W_gt = np.dot(np.dot(U, Sigma), np.transpose(V))
v_gt = np.asarray(v_gt)
train_x = []
train_y = []
for iter in range(n):
    train_x.append(np.random.normal(0.0,1.,d))
    train_y.append(np.dot(phi(np.dot(train_x[iter], W_gt)),v_gt))
train_x = np.asarray(train_x)
train_y = np.transpose(np.asarray(train_y))

In [ ]:
## moments
def gamma(j,sigma):
    estm = 0
    for i in range(10000):
        z = np.random.normal(0.0, 1.0)
        estm += phi_s(z*sigma)*(z**j)
    return estm/10000

m = np.zeros((4,k))
for i in range(k):
    m[0,i] = gamma(1,np.linalg.norm(W_gt[:,i]))
    m[1,i] = gamma(2,np.linalg.norm(W_gt[:,i])) - gamma(0,np.linalg.norm(W_gt[:,i]))
    m[2,i] = gamma(3,np.linalg.norm(W_gt[:,i])) - 3*gamma(1,np.linalg.norm(W_gt[:,i]))
    m[3,i] = gamma(4,np.linalg.norm(W_gt[:,i])) + 3*gamma(0,np.linalg.norm(W_gt[:,i])) - 6*gamma(2,np.linalg.norm(W_gt[:,i]))

In [ ]:
# m[2,3]/np.linalg.norm(W_gt[:,3])**2

In [ ]:
# M =  np.asarray(getM2(train_x, train_y))

# check = np.zeros((d,d))
# # check = np.zeros(d)
# for i in range(k):
#     check += v_gt[i]*m[1,i]*np.outer(W_gt[:,i], W_gt[:,i])
# #     check += v_gt[i]*m[0,i]*W_gt[:,i]
# print check
# print np.linalg.norm(M-check)/np.linalg.norm(check)
# print np.linalg.norm(M-check)/np.linalg.norm(M)

In [ ]:
def tensorInit(X, y):
    divInd = int(len(X)/3)
    
    # Partition
    X1 = X[:divInd]
    y1 = y[:divInd]

    X2 = X[divInd:2*divInd]
    y2 = y[divInd:2*divInd]
    
    X3 = X[2*divInd:]
    y3 = y[2*divInd:]
    
    ## P2 
    ## Estimating P2 as M2
    
    ## Power Method
    V = powMeth(k, X1, y1)
    
    ## R3
    R3 = multLnr2(getM3(X2, y2), V)
    
    #### KCL
    R = matlab.double(R3.flatten().tolist())
    eng = me.start_matlab()
    U = eng.notf_frompy(R, 100, k)
    eng.quit()
    U = np.asarray(U)
    #U = R3[0]

    ## RecMagSign
    return recmagsgn(V, U, X3, y3, m, W_gt) 

In [ ]:
X = train_x
y = train_y
divInd = int(len(X)/2)

# Partition
X1 = X[:divInd]
y1 = y[:divInd]

# X2 = X[divInd:2*divInd]
# y2 = y[divInd:2*divInd]

tensorWeights = tensorInit(train_x, train_y)
# tensorWeights = tensorInit(X1, y1)

In [ ]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=1.0)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    """
    h    = tf.square(tf.nn.relu((tf.matmul(X, w_1))))
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

In [ ]:
w_1 = init_weights((d, k))
w_1_random = tf.Variable(tf.cast(w_1, tf.float64))

In [ ]:
X = tf.placeholder("float64", shape=[None, d])
y = tf.placeholder("float64", shape=[None, 1])

# Weight initializations

# w_1 = w_1_random

w_1 = tf.Variable(np.transpose(tensorWeights[0]))

# w_2 = []
# v_choice = [1,-1]
# for iter in range(k):
#     w_2.append(random.choice(v_choice))
# w_2 = tf.cast(tf.Variable(np.transpose(np.asarray([w_2])), trainable=False), tf.float32)
# w_2 = init_weights((k, 1))

# w_2 = tf.Variable(tensorWeights[1], trainable=False)

w_2 = tf.Variable( np.transpose([v_gt]), trainable=False)
w_2 = tf.cast(w_2, tf.float64)

# Forward propagation
yhat  = forwardprop(X, w_1, w_2)

# Backward propagation
cost = tf.losses.mean_squared_error(y, yhat)
updates = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

# Run SGD
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
epsilon = 1e-4
x_e = []
y_tl3 = []
acc3 = []
T = 100
## Node Sparsity
s = 3
## Connection Sparsity
connection_thresh = 0.05
for epoch in range(T):
    
    i = 0
    
    ## Shuffling
    permList = np.random.permutation(range(n))
    train_x = train_x[permList]
    train_y = train_y[permList]
    
    for iter in range(int(n/batch_size)):
        sess.run(updates, feed_dict={X: train_x[i: i + batch_size], y: train_y[i: i + batch_size].reshape(batch_size,1)})
        i = (i + batch_size)%n
    train_accuracy = np.mean((train_y - sess.run(yhat, feed_dict={X:train_x})) <= epsilon)
    loss = sess.run(cost, feed_dict={X:train_x, y:train_y.reshape(n,1)})
    
    ## Iterative Hard Thresholding
    ## after every epoch
    
    ## Node Removal
    cur_w1 =  w_1.eval(session=sess)   
    normW1 = []
    for i in range(k):
        this_norm = np.linalg.norm(cur_w1[:,i])
        normW1.append((this_norm,i))
    normW1.sort()

    for i in range(s):
        cur_w1[:,normW1[i][1]] = np.zeros(d)
    assign_op = tf.assign(w_1, cur_w1)
    sess.run(assign_op)
    
    ## Connection Removal
#     cur_w1 =  w_1.eval(session=sess)   
#     for i in range(d):
#         for j in range(k):
#             this_norm = np.linalg.norm(cur_w1[i,j])
#             if this_norm <= connection_thresh:
#                 cur_w1[i,j] = 0.0
#     assign_op = tf.assign(w_1, cur_w1)
#     sess.run(assign_op)
    
    print "Epoch = ", epoch+1,"training loss: ", loss ," train Acc: ",100.*train_accuracy 
    x_e.append(epoch+1)
    y_tl3.append(loss)
    acc3.append(100.*train_accuracy)

In [ ]:
permList = list(itertools.permutations(range(k)))

# w_p = tensorWeights[0]
# w_p = np.random.normal(0, 0.1 , (k,d))
w_p = np.transpose(sess.run(w_1))
# v_p = tensorWeights[1]
v_p = sess.run(w_2)
temp = []
for i in v_p:
    temp.append(i[0])
v_p = np.asarray(temp)
for perm in permList:
    w_pi = w_p[list(perm)]
    v_pi = v_p[list(perm)]
    
    w_gt = np.transpose(W_gt)
    
    if sum(v_gt == v_pi) == k:
        max_diff = 0
        for i in range(k):
            diff = np.linalg.norm(w_pi[i]-w_gt[i])/np.linalg.norm(w_gt)
            if diff > max_diff:
                max_diff = diff
        if max_diff < 0.1:
            print "Success"
            break
    else:
        continue

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
NT = 50
y_tl = y_tl[:NT]
acc = acc[:NT]
x_e = x_e[:NT]

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x_e, y_tl0, 'blue', label="tensor")
# ax1.plot(x_e, y_tl, 'red', label="random")
ax1.plot(x_e, y_tl1, 'red', label="s=1")
ax1.plot(x_e, y_tl2, 'green', label="s=2")
ax1.plot(x_e, y_tl3, 'black', label="s=3")
# ax1.plot(x_e, y_tl4, 'magenta', label="s=4")
ax1.set_xlabel('epochs', color="white")
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('training loss', color='white')
ax1.tick_params('y', colors='white')
ax1.tick_params('x', colors='white')
l1 = ax1.legend(bbox_to_anchor=(1.1 ,0.5), loc=2, borderaxespad=0.)

# ax2 = ax1.twinx()
fig, ax2 = plt.subplots()
ax2.plot(x_e, acc0, 'blue', label="tensor_acc")
# ax2.plot(x_e, acc, 'red', label="random_acc")
ax2.plot(x_e, acc1, 'red',label="s=1_acc")
ax2.plot(x_e, acc2, 'green',label="s=2_acc")
ax2.plot(x_e, acc3, 'black',label="s=3_acc")
# ax2.plot(x_e, acc4, 'magenta',label="s=4_acc")
ax2.set_ylabel('accuracy', color='white')
ax2.tick_params('y', colors='white')
ax2.tick_params('x', colors='white')
ax2.set_xlabel('epochs', color="white")
l2 = ax2.legend(bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.)
# ax2.set_ylim([40.0,60.0])

# fig.tight_layout()
plt.show()